In [1]:
# Get the length if npy
import sys
import os
import numpy as np
def get_npy_length(filename):
    if not os.path.exists(filename):
        raise FileNotFoundError(f"File {filename} does not exist.")
    try:
        data = np.load(filename, allow_pickle=True)
        return len(data)
    except Exception as e:
        raise RuntimeError(f"Error loading {filename}: {e}")
print(get_npy_length("data/0/2_labels.npy"))
print(get_npy_length("output/0/labels/2.npy"))

5516
3028


In [19]:
import os
import numpy as np
# import open3d as o3d

# ─── CONFIG ────────────────────────────────────────────────────────────────
output_dir   = 'output'                 # root of your data
results_root ='result'
velo_root    = output_dir               # 'output/<seq>/velodyne/<i>.npy'

# define your color map for labels
label_to_color = {
    0: (0.0, 0.0, 0.0),  # background → black
    1: (0.0, 1.0, 0.0),  # class 1  → green
    2: (1.0, 0.0, 0.0),  # class 2  → red
    # … add more if you have more classes
}

# ─── PROCESS EACH SEQUENCE ─────────────────────────────────────────────────
for seq in sorted(os.listdir(results_root)):

    seq_pred_dir = os.path.join(results_root, seq, 'predictions')
    seq_velo_dir      = os.path.join(output_dir, seq, 'velodyne')
    seq_original_dir  = os.path.join(output_dir, seq, 'labels')

    # # iterate frames in numeric order
    for fname in sorted(os.listdir(seq_pred_dir), key=lambda x: int(os.path.splitext(x)[0])):
        
        if not fname.endswith('.npy'):
            continue
        # paths
        pred_path  = os.path.join(seq_pred_dir,    fname)
        label_path = os.path.join(seq_original_dir, fname)

        pred = np.load(pred_path)    # e.g. shape (3067,)
        orig = np.load(label_path)   # e.g. shape (3067,1) or (3067,)

        # remember the original shape & dtype
        orig_shape = orig.shape
        orig_dtype = orig.dtype

        # flatten both
        pred_flat = pred.ravel()     # shape (N,)
        orig_flat = orig.ravel()     # shape (N,)

        # sanity‐check size
        if pred_flat.size != orig_flat.size:
            print(f"⚠ size mismatch in {seq}/{fname}: "
                  f"pred_flat.size={pred_flat.size}, orig_flat.size={orig_flat.size} → skipping")
            continue

        # mask‐overwrite
        adjusted_flat = pred_flat.copy()
        mask = (orig_flat != 0)
        adjusted_flat[mask] = orig_flat[mask]

        # reshape back to original GT shape & dtype
        adjusted = adjusted_flat.reshape(orig_shape).astype(orig_dtype)
        # print(adjusted.shape, orig.shape)
        # overwrite the original label file in-place
        np.save(label_path, adjusted)
    print(f"Processed sequence {seq} with {len(os.listdir(seq_pred_dir))} frames.")
        # save adjusted
        # # out_path = os.path.join(seq_adjusted_dir, fname)
        # # np.save(out_path, adjusted)
        # # print(f"  ✔ adjusted saved: {out_path}")

        # # ─── VISUALIZE ─────────────────────────────────────────────────────
        # points = np.load(velo_path)          # shape (N,4)
        # labels = adjusted.astype(int).ravel()# shape (N,)

        # xyz = points[:, :3]
        # pcd = o3d.geometry.PointCloud()
        # pcd.points = o3d.utility.Vector3dVector(xyz)

        # # apply color mapping
        # colors = np.array([ label_to_color.get(lbl, (0.5,0.5,0.5)) 
        #                     for lbl in gt ], dtype=np.float64)
        # pcd.colors = o3d.utility.Vector3dVector(colors)

        # o3d.visualization.draw_geometries([pcd])
        # print(f"Visualized frame {frame_id} with original labels.")
        # # visualuize adjusted
        # adjusted_colors = np.array([ label_to_color.get(lbl, (0.5,0.5,0.5)) 
        #                              for lbl in labels ], dtype=np.float64)
        # pcd.colors = o3d.utility.Vector3dVector(adjusted_colors)
        # o3d.visualization.draw_geometries([pcd])
        # print(f"Visualized frame {frame_id} with adjusted labels.")


Processed sequence 0 with 93 frames.
Processed sequence 1 with 167 frames.
Processed sequence 2 with 1363 frames.
Processed sequence 3 with 436 frames.
Processed sequence 4 with 291 frames.
Processed sequence 5 with 44 frames.
Processed sequence 6 with 1170 frames.
Processed sequence 7 with 1545 frames.


In [ ]:
# An